In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Sample movie data
movies = pd.DataFrame({
    'movie_id': [1, 2, 3, 4, 5],
    'title': ['Toy Story', 'Jumanji', 'Grumpier Old Men', 'Waiting to Exhale', 'Father of the Bride Part II'],
    'genres': ['Animation|Children|Comedy', 'Adventure|Children|Fantasy', 'Comedy|Romance', 'Comedy|Drama|Romance', 'Comedy']
})

# Sample user ratings
ratings = pd.DataFrame({
    'user_id': [1, 1, 1, 2, 2, 2, 3, 3, 3],
    'movie_id': [1, 2, 3, 1, 2, 4, 2, 3, 5],
    'rating': [5, 3, 4, 4, 2, 5, 3, 4, 2]
})

# Collaborative Filtering Recommendation
def collaborative_filtering_recommendation(user_id, ratings, movies, n_recommendations=3):
    user_ratings = ratings.pivot(index='user_id', columns='movie_id', values='rating')
    user_similarity = cosine_similarity(user_ratings.fillna(0))
    similar_users = pd.DataFrame(user_similarity, index=user_ratings.index, columns=user_ratings.index)
    
    # Find similar users
    similar_users = similar_users[user_id].sort_values(ascending=False)
    similar_user_ids = similar_users.index[1:n_recommendations + 1]

    # Recommend movies rated by similar users
    recommended_movies = []
    for other_user in similar_user_ids:
        recommended_movies += ratings[ratings['user_id'] == other_user]['movie_id'].tolist()

    # Filter out movies already rated by the user
    already_rated = ratings[ratings['user_id'] == user_id]['movie_id'].tolist()
    recommended_movies = list(set(recommended_movies) - set(already_rated))

    return movies[movies['movie_id'].isin(recommended_movies)]['title'].tolist()

# Content-Based Filtering Recommendation
def content_based_filtering_recommendation(user_id, ratings, movies, n_recommendations=3):
    # Merge ratings and movies
    user_data = ratings[ratings['user_id'] == user_id].merge(movies, on='movie_id')

    # Use TF-IDF Vectorizer on genres
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(movies['genres'])

    # Compute cosine similarity
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Get indices of movies the user has rated
    movie_indices = user_data['movie_id'].tolist()

    # Get scores for similar movies
    sim_scores = list(enumerate(cosine_sim[movie_indices].mean(axis=0)))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get movie recommendations
    recommended_movie_indices = [i for i, _ in sim_scores if i not in movie_indices][:n_recommendations]
    
    return movies.iloc[recommended_movie_indices]['title'].tolist()

# Test the recommendation systems
user_id = 1
print("Collaborative Filtering Recommendations:")
print(collaborative_filtering_recommendation(user_id, ratings, movies))

print("\nContent-Based Filtering Recommendations:")
print(content_based_filtering_recommendation(user_id, ratings, movies))

Collaborative Filtering Recommendations:
['Waiting to Exhale', 'Father of the Bride Part II']

Content-Based Filtering Recommendations:
['Father of the Bride Part II', 'Toy Story']
